# Actividad: Regresión Líneal Tokio

In [2451]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2452]:
# Cargar archivo CSV usando pandas
df = pd.read_csv('Tokio_Limpio.csv')
df.head(2)

,Unnamed: 0.1,last_scraped,name,description,neighborhood_overview,host_location,host_response_time,host_is_superhost,host_verifications,host_has_profile_pic,...,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,0,2024-09-27,Oshiage Holiday Apartment,?,?,"Tokyo, Japan",within a day,t,"['email', 'phone']",t,...,22.3,7.0,1.0,4.78,4.91,4.84,4.83,4.55,4.80,1.14
1,1,2024-09-27,Kero-kero house room 1,We have been in airbnb since 2011 and it has g...,We love Nishinippori because is nearer to Toky...,Japan,within an hour,t,"['email', 'phone']",t,...,22.3,10.0,2.0,4.98,4.91,4.98,4.98,4.84,4.92,1.79


## Creación de los dataframes con las variables a utilizar

In [2453]:
# Convertimos las variables price y review_scores_rating a numérico
df['price'] = pd.to_numeric(df['price'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')
df['review_scores_rating'] = pd.to_numeric(df['review_scores_rating'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')

In [2454]:
# Rellenamos los valores NaN con bfill
df['price'].fillna(method='bfill', inplace=True)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3508\3229511809.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(method='bfill', inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3508\3229511809.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['price'].fillna(method='bfill', inplace=True)


In [2455]:
# Cambiamos los valores '?' el valor que mas se repite en la columna (t o f) para la columna host_is_superhost
df['host_is_superhost'] = df['host_is_superhost'].replace({'?': df['host_is_superhost'].mode()[0]})
df['has_availability'] = df['has_availability'].replace({'?': df['has_availability'].mode()[0]})

In [2456]:
# Creamos una copia del DataFrame para las variables dependientes: host_is_superhost, host_has_profile_pic, host_identity_verified, has_availability, instant_bookable, price, property_type, accommodates, room_type, review_scores_rating.
df_Dep = df[['host_is_superhost', 'host_has_profile_pic', 
             'host_identity_verified', 'has_availability', 
             'instant_bookable', 'price', 'property_type', 
             'accommodates', 'room_type', 'review_scores_rating']].copy()
df_Dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18736 entries, 0 to 18735
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   host_is_superhost       18736 non-null  object 
 1   host_has_profile_pic    18736 non-null  object 
 2   host_identity_verified  18736 non-null  object 
 3   has_availability        18736 non-null  object 
 4   instant_bookable        18736 non-null  object 
 5   price                   18736 non-null  float64
 6   property_type           18736 non-null  object 
 7   accommodates            18736 non-null  float64
 8   room_type               18736 non-null  object 
 9   review_scores_rating    18736 non-null  float64
dtypes: float64(3), object(7)
memory usage: 1.4+ MB


In [2457]:
# Convertimos instant_bookable a binario
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})

In [2458]:
# Creamos una copia del DataFrame para las variables independientes: host_response_rate, number_of_reviews, review_scores_rating, review_scores_communication, availability_365, minimum_nights, maximum_nights, availability_30, accommodates, bedrooms, bathrooms, price, review_scores_cleanliness, review_scores_value, instant_bookable.
df_Ind = df[['host_response_rate', 'number_of_reviews',
             'review_scores_rating', 'review_scores_communication', 
             'availability_365', 'minimum_nights', 'maximum_nights', 
             'availability_30', 'accommodates', 'bedrooms', 
             'bathrooms', 'beds', 'price', 'review_scores_cleanliness', 
             'review_scores_value', 'instant_bookable']].copy()
df_Ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18736 entries, 0 to 18735
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_response_rate           18736 non-null  float64
 1   number_of_reviews            18736 non-null  float64
 2   review_scores_rating         18736 non-null  float64
 3   review_scores_communication  18736 non-null  float64
 4   availability_365             18736 non-null  int64  
 5   minimum_nights               18736 non-null  float64
 6   maximum_nights               18736 non-null  float64
 7   availability_30              18736 non-null  float64
 8   accommodates                 18736 non-null  float64
 9   bedrooms                     18736 non-null  float64
 10  bathrooms                    18736 non-null  float64
 11  beds                         18736 non-null  float64
 12  price                        18736 non-null  float64
 13  review_scores_cl

## Convertir a dicótmocas algunas variables dependientes

In [2459]:
# Convertimos la variable price a dicotómica a partir de su promedio 
df_Dep['price'] = df_Dep['price'].apply(lambda x: 1 if x > df_Dep['price'].mean() else 0)
df_Dep['price'] = df_Dep['price'].replace({1: 'higher', 0: 'lower'})
df_Dep['price'].value_counts()

price
lower     12872
higher     5864
Name: count, dtype: int64

In [2460]:
# Convertimos la variable review_scores_rating a dicotómica a partir de su promedio
df_Dep['review_scores_rating'] = df_Dep['review_scores_rating'].apply(lambda x: 1 if x > df_Dep['review_scores_rating'].mean() else 0)
df_Dep['review_scores_rating'] = df_Dep['review_scores_rating'].replace({1: 'higher', 0: 'lower'}) 
df_Dep['review_scores_rating'].value_counts()

review_scores_rating
higher    10985
lower      7751
Name: count, dtype: int64

In [2461]:
# Convertimos la variable accommodates a dicotómica a partir de su promedio
df_Dep['accommodates'] = df_Dep['accommodates'].apply(lambda x: 1 if x > df_Dep['accommodates'].mean() else 0)
df_Dep['accommodates'] = df_Dep['accommodates'].replace({1: 'higher', 0: 'lower'})
df_Dep['accommodates'].value_counts()

accommodates
lower     12896
higher     5840
Name: count, dtype: int64

In [2462]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['property_type'])
df_Dep['property_type'].value_counts()

property_type
Entire rental unit             11288
Entire home                     2382
Room in hotel                    882
Private room in home             730
Private room in rental unit      684
                               ...  
Private room in barn               1
Entire bungalow                    1
Treehouse                          1
Private room in camper/rv          1
Shipping container                 1
Name: count, Length: 64, dtype: int64

In [2463]:
# Convertimos la variable property_type a dicotómica con Entire rental unit y el resto other
df_Dep['property_type'] = df_Dep['property_type'].apply(lambda x: 1 if x == 'Entire rental unit' else 0)
df_Dep['property_type'] = df_Dep['property_type'].replace({1: 'entire', 0: 'other'})
df_Dep['property_type'].value_counts()

property_type
entire    11288
other      7448
Name: count, dtype: int64

In [2464]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
df_Dep['room_type'].value_counts()

room_type
Entire home/apt    15109
Private room        3058
Shared room          424
Hotel room           145
Name: count, dtype: int64

In [2465]:
# Convertimos la variable room_type a dicotómica con Entire rental unit y el resto other
df_Dep['room_type'] = df_Dep['room_type'].apply(lambda x: 1 if x == 'Entire home/apt' else 0)
df_Dep['room_type'] = df_Dep['room_type'].replace({1: 'apt', 0: 'other'})
df_Dep['room_type'].value_counts()

room_type
apt      15109
other     3627
Name: count, dtype: int64

## Regresión líneal de las variables

### host_is_superhost

In [2466]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['host_is_superhost']

In [2467]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2468]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2469]:
#se escalan todos los datos
escalar = StandardScaler()

In [2470]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2471]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2472]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2473]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 't', 'f', ..., 't', 'f', 'f'], dtype=object)

In [2474]:
unico = np.unique(df_Dep['host_is_superhost'])
unico

array(['f', 't'], dtype=object)

In [2475]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3246  300]
 [1389  686]]


In [2476]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.6957403651115619


In [2477]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.6995196584237681


In [2478]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9153976311336718


### host_has_profile_pic

In [2479]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_communication']]
Vars_Dep = df_Dep['host_has_profile_pic']

In [2480]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2481]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2482]:
#se escalan todos los datos
escalar = StandardScaler()

In [2483]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2484]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2485]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2486]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2487]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0   81]
 [   0 5540]]


In [2488]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9855897527130404


In [2489]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9855897527130404


In [2490]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### host_identity_verified

In [2491]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'availability_365']]
Vars_Dep = df_Dep['host_identity_verified']

In [2492]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2493]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2494]:
#se escalan todos los datos
escalar = StandardScaler()

In [2495]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2496]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2497]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2498]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2499]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  173]
 [   0 5448]]


In [2500]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9692225582636541


In [2501]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9692225582636541


In [2502]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### has_availability

In [2503]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['minimum_nights', 'maximum_nights', 'availability_30', 'availability_365']]
Vars_Dep = df_Dep['has_availability']

In [2504]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2505]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2506]:
#se escalan todos los datos
escalar = StandardScaler()

In [2507]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2508]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2509]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2510]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2511]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0    1]
 [   0 5620]]


In [2512]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9998220957125067


In [2513]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9998220957125067


In [2514]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### instant_bookable

In [2515]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['instant_bookable']

In [2516]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2517]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2518]:
#se escalan todos los datos
escalar = StandardScaler()

In [2519]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2520]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2521]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2522]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2523]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0 1400]
 [   0 4221]]


In [2524]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.75093399750934


In [2525]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.75093399750934


In [2526]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### price

In [2527]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['price']

In [2528]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2529]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2530]:
#se escalan todos los datos
escalar = StandardScaler()

In [2531]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2532]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2533]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2534]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['lower', 'lower', 'lower', ..., 'lower', 'higher', 'lower'],
      dtype=object)

In [2535]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 772 1053]
 [ 311 3485]]


In [2536]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.7128347183748845


In [2537]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.7573385518590998


In [2538]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9180716543730242


### property_type

In [2539]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'price', 'review_scores_rating']]
Vars_Dep = df_Dep['property_type']

In [2540]:
# Verificamos que las variables independientes no tengan valores nulos
print(Vars_Indep.isnull().sum())

accommodates            0
bedrooms                0
price                   0
review_scores_rating    0
dtype: int64


In [2541]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2542]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2543]:
#se escalan todos los datos
escalar = StandardScaler()

In [2544]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2545]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2546]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2547]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['entire', 'entire', 'entire', ..., 'entire', 'entire', 'other'],
      dtype=object)

In [2548]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2847  524]
 [1581  669]]


In [2549]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="entire")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.6429539295392954


In [2550]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.6255114748265433


In [2551]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="other")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.29733333333333334


### accommodates

In [2552]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['bedrooms', 'bathrooms', 'beds', 'price', 'number_of_reviews']]
Vars_Dep = df_Dep['accommodates']

In [2553]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2554]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2555]:
#se escalan todos los datos
escalar = StandardScaler()

In [2556]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2557]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2558]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2559]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['lower', 'higher', 'lower', ..., 'lower', 'lower', 'lower'],
      dtype=object)

In [2560]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1176  595]
 [ 337 3513]]


In [2561]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.7772637144745539


In [2562]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8341932040562178


In [2563]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9124675324675324


### room_type

In [2564]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'price', 'instant_bookable']]
Vars_Dep = df_Dep['room_type']

In [2565]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2566]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2567]:
#se escalan todos los datos
escalar = StandardScaler()

In [2568]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2569]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2570]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2571]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['apt', 'apt', 'apt', ..., 'apt', 'apt', 'apt'], dtype=object)

In [2572]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4479   16]
 [1033   93]]


In [2573]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="apt")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8125907111756169


In [2574]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8133784024194983


In [2575]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="other")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.08259325044404973


### review_scores_rating

In [2576]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['number_of_reviews', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_value']]
Vars_Dep = df_Dep['review_scores_rating']

In [2577]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2578]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2579]:
#se escalan todos los datos
escalar = StandardScaler()

In [2580]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2581]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2582]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2583]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['higher', 'lower', 'higher', ..., 'higher', 'lower', 'lower'],
      dtype=object)

In [2584]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2956  414]
 [ 476 1775]]


In [2585]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8613053613053613


In [2586]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8416651841309376


In [2587]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.7885384273656153
